In [519]:


import datetime
import pandas as pd
from tensorflow.python.keras.layers.core import Dropout
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re 
import matplotlib.pyplot as plt
import random
from pandas_datareader import data
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
import math
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 1500)



In [520]:
news_data = pd.read_excel("newsDb.xlsx") 
# print(news_data.head())
# print(news_data.head())
historical_data = pd.read_excel("finaceDbTSLA.xlsx")
# print(historical_data.head())

merged_data = news_data.merge(historical_data, left_on='dateTime', right_on='Date') #merge two data frame
# print(merged_data.head())#show a couples of merged data.
merged_data.to_excel("deneme.xlsx")

In [521]:

headlines= []
for row in range(0, len(merged_data.index)):
    headlines.append(' '.join(str(x) for x in merged_data.iloc[row, 1:11]))
# print(headlines[0])

clean_headlines = []
for i in range(0, len(headlines)):
  clean_headlines.append(re.sub("b[(')]", '', headlines[i]))
  clean_headlines[i] = re.sub('b[(")]', '', headlines[i])
  clean_headlines[i] = re.sub("\'", '', headlines[i]) #replace ' && " with nulls.
# print(clean_headlines[0])
# print(len(clean_headlines))  #headline içerisinde nan yazıları kaldırılmalı!

merged_data['Combined_News'] = clean_headlines

In [522]:

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity 

In [523]:

merged_data['Subjectivity']=merged_data['article'].apply(getSubjectivity)
merged_data['Polarity']=merged_data['article'].apply(getPolarity)
print(merged_data)


      dateTime                                                 H1  \
0   2021-12-14  California senator scrutinizing Tesla’s Full S...   
1   2021-12-13  Ticker: Time names Elon Musk “Person of the Ye...   
2   2021-12-10  North Carolina chases more success at home vs....   
3   2021-12-09  Drivers playing video games? U.S. is looking i...   
4   2021-12-08  BofA Explains Neutral Rating for Tesla, Buy fo...   
5   2021-12-07  Tesla's Musk says Biden's electric vehicle bil...   
6   2021-12-06  Interview With Cassava Sciences' CEO Remi Barb...   
7   2021-12-03  Robeco Institutional Asset Management B.V. Low...   
8   2021-12-02  Mirrored Tesla Tops 24 Hour Trading Volume of ...   
9   2021-12-01  Tesla officially moves headquarters from Calif...   
10  2021-11-30  Cramer's Mad Money Recap 11/30: Amazon, Micros...   
11  2021-11-29  Elon Musk’s California exit can save him $2 bi...   
12  2021-11-26  Great Valley Advisor Group Inc. Has $4.10 Mill...   
13  2021-11-24  Musk sells more th

In [524]:
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

In [525]:
compound=[]
SIA=0
for i in range(0,len(merged_data['Combined_News'])):
    SIA = getSIA(merged_data['Combined_News'][i])
    compound.append(SIA['compound'])

In [526]:
merged_data['Compound']=compound


In [527]:
merged_data.to_excel('merged_data_tesla4.xlsx')


In [528]:
keep_columns=['Open','High','Low','Volume','Subjectivity','Polarity','Compound']


In [529]:
y_values = np.array(merged_data['Close'])


In [530]:
y_values

array([ 958.51001 ,  966.409973, 1017.030029, 1003.799988, 1068.959961,
       1051.75    , 1009.01001 , 1014.969971, 1084.599976, 1095.      ,
       1144.76001 , 1136.98999 , 1081.920044, 1116.      , 1109.030029,
       1156.869995, 1137.060059, 1096.380005, 1089.01001 , 1054.72998 ,
       1013.390015, 1033.420044, 1063.51001 , 1067.949951, 1023.5     ,
       1162.939941, 1222.089966, 1229.910034, 1213.859985, 1172.      ,
       1208.589966, 1114.      , 1077.040039, 1037.859985, 1018.429993,
       1024.859985,  909.679993,  894.      ,  865.799988,  864.27002 ,
        870.109985,  843.030029,  818.320007,  811.080017,  805.719971,
        791.940002,  785.48999 ,  793.609985,  782.75    ,  780.590027,
        781.530029,  775.219971,  775.47998 ,  781.309998,  777.559998,
        791.359985,  774.390015,  753.640015,  751.940002,  739.380005,
        730.169983,  759.48999 ,  756.98999 ,  755.830017,  744.48999 ,
        743.      ,  736.27002 ,  754.859985,  753.869995,  752.

In [531]:
x_values=merged_data[keep_columns].values #numpy array.

In [532]:
x_values

array([[ 9.45000000e+02,  9.66409973e+02,  9.30000000e+02,
         2.35330000e+07,  3.57913165e-01,  1.68137255e-01,
        -9.34900000e-01],
       [ 1.00109003e+03,  1.00500000e+03,  9.51419983e+02,
         2.61985000e+07,  6.50000000e-01,  1.03409091e-01,
        -3.19700000e-01],
       [ 1.00875000e+03,  1.02097998e+03,  9.82530029e+02,
         1.98550000e+07,  5.95344828e-01,  2.19166667e-01,
         9.04100000e-01],
       [ 1.06064001e+03,  1.06248999e+03,  1.00235999e+03,
         1.98128000e+07,  4.44949495e-01,  7.24747475e-02,
         9.41800000e-01],
       [ 1.05270996e+03,  1.07238000e+03,  1.03300000e+03,
         1.39688000e+07,  4.73778040e-01,  1.56544667e-01,
         7.90600000e-01],
       [ 1.04419995e+03,  1.05767004e+03,  1.02681006e+03,
         1.86949000e+07,  5.82659933e-01,  2.66835017e-01,
         8.88500000e-01],
       [ 1.00151001e+03,  1.02164001e+03,  9.50500000e+02,
         2.72210000e+07,  2.67500000e-01, -5.25000000e-02,
        -8.9640000

In [533]:

training_size=int(len(x_values)*0.75)
test_size=len(y_values)-training_size
x_train,x_test=x_values[0:training_size,:],x_values[training_size:len(x_values),:]
y_train, y_test = y_values[0:training_size], y_values[training_size:len(y_values)]

In [534]:
y_train

array([ 958.51001 ,  966.409973, 1017.030029, 1003.799988, 1068.959961,
       1051.75    , 1009.01001 , 1014.969971, 1084.599976, 1095.      ,
       1144.76001 , 1136.98999 , 1081.920044, 1116.      , 1109.030029,
       1156.869995, 1137.060059, 1096.380005, 1089.01001 , 1054.72998 ,
       1013.390015, 1033.420044, 1063.51001 , 1067.949951, 1023.5     ,
       1162.939941, 1222.089966, 1229.910034, 1213.859985, 1172.      ,
       1208.589966, 1114.      , 1077.040039, 1037.859985, 1018.429993,
       1024.859985,  909.679993,  894.      ,  865.799988,  864.27002 ,
        870.109985,  843.030029,  818.320007,  811.080017,  805.719971,
        791.940002,  785.48999 ,  793.609985,  782.75    ,  780.590027,
        781.530029,  775.219971])

In [535]:
y_test

array([775.47998 , 781.309998, 777.559998, 791.359985, 774.390015,
       753.640015, 751.940002, 739.380005, 730.169983, 759.48999 ,
       756.98999 , 755.830017, 744.48999 , 743.      , 736.27002 ,
       754.859985, 753.869995, 752.919983])

In [536]:
print(x_train.shape, x_test.shape, y_train.shape)

(52, 7) (18, 7) (52,)


In [537]:
#scaling
scaler = MinMaxScaler(feature_range=(0,1))
x_train=scaler.fit_transform(x_train) # hem fit ediyor hem de scale ediyor. //prediction gerceklesiyor.
x_test=scaler.transform(x_test) # sadece transform olacak. cünkü model bu veriyi eğitim esnasında görmeyecek //prediction uygulayacak.


In [538]:
y_train = np.array([y_train]).T
y_test = np.array([y_test]).T

In [539]:
y_train.shape

(52, 1)

In [540]:
scaler2 = StandardScaler()
y_train = scaler2.fit_transform(y_train)
y_test = scaler2.transform(y_test)

In [541]:
y_train

array([[-0.33889434],
       [-0.28105011],
       [ 0.08959442],
       [-0.00727711],
       [ 0.46982998],
       [ 0.34381712],
       [ 0.03087113],
       [ 0.07451049],
       [ 0.58434755],
       [ 0.66049745],
       [ 1.02484465],
       [ 0.96795187],
       [ 0.56472485],
       [ 0.81426131],
       [ 0.76322656],
       [ 1.11351503],
       [ 0.96846492],
       [ 0.67060197],
       [ 0.61663821],
       [ 0.3656368 ],
       [ 0.06294191],
       [ 0.20960356],
       [ 0.42992495],
       [ 0.46243459],
       [ 0.13696812],
       [ 1.15795971],
       [ 1.59106143],
       [ 1.64832066],
       [ 1.53080078],
       [ 1.22429826],
       [ 1.49221323],
       [ 0.79961713],
       [ 0.52899302],
       [ 0.2421132 ],
       [ 0.09984508],
       [ 0.14692605],
       [-0.69643205],
       [-0.81124234],
       [-1.01772533],
       [-1.02892789],
       [-0.98616715],
       [-1.18444898],
       [-1.36537795],
       [-1.4183898 ],
       [-1.45763653],
       [-1

In [542]:
y_test

array([[-1.67905642],
       [-1.63636851],
       [-1.66382635],
       [-1.56278162],
       [-1.68703724],
       [-1.83897058],
       [-1.85141822],
       [-1.94338363],
       [-2.01082023],
       [-1.79613654],
       [-1.81444177],
       [-1.82293519],
       [-1.90596787],
       [-1.91687771],
       [-1.96615522],
       [-1.83003785],
       [-1.83728664],
       [-1.84424272]])

In [543]:
'''
sc2 = MinMaxScaler()
y_train = sc2.fit_transform(y_train.reshape(-1,1))
'''

'\nsc2 = MinMaxScaler()\ny_train = sc2.fit_transform(y_train.reshape(-1,1))\n'

In [544]:
y_test

array([[-1.67905642],
       [-1.63636851],
       [-1.66382635],
       [-1.56278162],
       [-1.68703724],
       [-1.83897058],
       [-1.85141822],
       [-1.94338363],
       [-2.01082023],
       [-1.79613654],
       [-1.81444177],
       [-1.82293519],
       [-1.90596787],
       [-1.91687771],
       [-1.96615522],
       [-1.83003785],
       [-1.83728664],
       [-1.84424272]])

In [546]:
timestep = 1

In [547]:
x_train = x_train.reshape((len(x_train), timestep, len(x_train[0])))

In [548]:
x_test = x_test.reshape((len(x_test), timestep, len(x_test[0])))

In [549]:
print(x_test.shape)

(18, 1, 7)


In [550]:
print(x_train.shape)

(52, 1, 7)


In [551]:

model=Sequential()
model.add(LSTM(units = 50, input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

# model.add(LSTM(units=50,return_sequences=True))
# model.add(LSTM(units=50))
model.add(Dense(units = 1))


In [552]:
model.compile(optimizer='adam',loss='mean_squared_error')


In [553]:
model.fit(x_train, y_train, epochs=100,batch_size=8,verbose=1) # epochs=100


Epoch 1/100
7/7 [==============================] - 3s 5ms/step - loss: 0.9358
Epoch 2/100
7/7 [==============================] - 0s 5ms/step - loss: 0.9100
Epoch 3/100
7/7 [==============================] - 0s 6ms/step - loss: 0.8714
Epoch 4/100
7/7 [==============================] - 0s 5ms/step - loss: 0.8595
Epoch 5/100
7/7 [==============================] - 0s 5ms/step - loss: 0.8244
Epoch 6/100
7/7 [==============================] - 0s 5ms/step - loss: 0.7932
Epoch 7/100
7/7 [==============================] - 0s 5ms/step - loss: 0.7639
Epoch 8/100
7/7 [==============================] - 0s 5ms/step - loss: 0.7303
Epoch 9/100
7/7 [==============================] - 0s 5ms/step - loss: 0.7138
Epoch 10/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6720
Epoch 11/100
7/7 [==============================] - 0s 5ms/step - loss: 0.6262
Epoch 12/100
7/7 [==============================] - 0s 5ms/step - loss: 0.5930
Epoch 13/100
7/7 [==============================] - 0s 5ms/st

In [554]:
train_predict=model.predict(x_train)
test_predict=model.predict(x_test)

train_predict = train_predict.reshape((train_predict.shape[0], train_predict.shape[2]))

test_predict = test_predict.reshape((test_predict.shape[0], test_predict.shape[2]))


print('y_train shape: ', y_train.shape, '\ny_train', y_train[0:5], '\n')
print('y_train_pred shape: ', train_predict.shape, '\ny_train_pred', train_predict[0:5], '\n')
print('y_test shape: ', y_test.shape, '\ny_test', y_test[0:5], '\n')
print('y_test_pred shape: ', test_predict.shape, '\ny_test_pred', test_predict[0:5])

y_train shape:  (52, 1) 
y_train [[-0.33889434]
 [-0.28105011]
 [ 0.08959442]
 [-0.00727711]
 [ 0.46982998]] 

y_train_pred shape:  (52, 1) 
y_train_pred [[-0.3476145 ]
 [-0.11565737]
 [-0.10555497]
 [ 0.24026994]
 [ 0.3185392 ]] 

y_test shape:  (18, 1) 
y_test [[-1.67905642]
 [-1.63636851]
 [-1.66382635]
 [-1.56278162]
 [-1.68703724]] 

y_test_pred shape:  (18, 1) 
y_test_pred [[-1.3834532]
 [-1.663215 ]
 [-1.3524636]
 [-1.4734936]
 [-1.3161525]]


In [555]:
test_predict = scaler2.inverse_transform(test_predict)

In [556]:
train_predict = scaler2.inverse_transform(train_predict)

In [557]:
y_train = scaler2.inverse_transform(y_train)
y_test = scaler2.inverse_transform(y_test)

In [558]:
rmse = np.sqrt(mean_squared_error(y_test, test_predict))

In [559]:
rmse

27.070171553716126

In [560]:
mape_test = mean_absolute_percentage_error(y_test, test_predict)
mape_train = mean_absolute_percentage_error(y_train, train_predict)

In [561]:
mape_test

0.027580833476410545

In [562]:
mape_train

0.018417237804805177

In [563]:
y_test

array([[775.47998 ],
       [781.309998],
       [777.559998],
       [791.359985],
       [774.390015],
       [753.640015],
       [751.940002],
       [739.380005],
       [730.169983],
       [759.48999 ],
       [756.98999 ],
       [755.830017],
       [744.48999 ],
       [743.      ],
       [736.27002 ],
       [754.859985],
       [753.869995],
       [752.919983]])

In [564]:
test_predict

array([[815.8514 ],
       [777.6435 ],
       [820.08374],
       [803.5543 ],
       [825.0429 ],
       [762.55225],
       [800.4791 ],
       [759.5673 ],
       [745.55975],
       [811.9298 ],
       [750.8806 ],
       [778.2676 ],
       [752.2301 ],
       [745.9977 ],
       [755.60443],
       [764.1922 ],
       [758.4867 ],
       [742.7885 ]], dtype=float32)

In [565]:
train_predict

array([[ 957.3191 ],
       [ 988.99817],
       [ 990.37787],
       [1037.6083 ],
       [1048.2977 ],
       [1031.8925 ],
       [1006.3755 ],
       [1052.6243 ],
       [1090.2898 ],
       [1134.5465 ],
       [1137.9972 ],
       [1121.0647 ],
       [1090.8904 ],
       [1095.1398 ],
       [1131.4231 ],
       [1167.8832 ],
       [1114.6683 ],
       [1096.9803 ],
       [1079.4563 ],
       [1020.79364],
       [1011.951  ],
       [1032.284  ],
       [1083.9196 ],
       [1022.52014],
       [1110.0579 ],
       [1158.6978 ],
       [1236.1758 ],
       [1218.7855 ],
       [1192.8867 ],
       [1175.6732 ],
       [1164.5156 ],
       [1092.3373 ],
       [1064.3859 ],
       [1045.7672 ],
       [1034.7633 ],
       [ 967.4036 ],
       [ 883.68414],
       [ 900.3133 ],
       [ 872.5355 ],
       [ 870.7556 ],
       [ 853.4138 ],
       [ 846.62665],
       [ 811.487  ],
       [ 804.5901 ],
       [ 798.6906 ],
       [ 804.46497],
       [ 784.4006 ],
       [ 797.